<a href="https://colab.research.google.com/github/hoohahaBIGHEAD/AI/blob/main/%EB%94%B0%EB%A6%89%EC%9D%B4_%EB%8C%80%EC%97%AC%EC%88%98_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 라이브러리 및 데이터
##Library & Data

In [1]:
!pip install pycaret
!pip install optuna


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

##데이터 로딩 및 확인

In [3]:
bike_data = pd.read_csv('/content/drive/MyDrive/tmp/따릉이/train.csv')
bike_data.drop(['id'], axis = 1, inplace=True)
test = pd.read_csv('/content/drive/MyDrive/tmp/따릉이/test.csv')
test.drop(['id'], axis = 1, inplace=True)
submission = pd.read_csv('/content/drive/MyDrive/tmp/따릉이/submission.csv')

결측치 처리

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

it_train = bike_data.copy()
it_train = IterativeImputer(random_state=1).fit_transform(it_train)

itImp = pd.DataFrame(it_train)
itImp.columns = bike_data.columns

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:670: ConvergenceWarning:

[IterativeImputer] Early stopping criterion not reached.



이상치 처리

In [7]:
itImp_mid = itImp.copy()

col_name = bike_data.columns.drop(['hour_bef_precipitation'])
for iIt in col_name:
  Q1=itImp_mid[iIt].quantile(0.25)
  Q3=itImp_mid[iIt].quantile(0.75)
  IQR = Q3-Q1
  train_delout = itImp_mid[(itImp_mid[iIt]<(Q1 - 1.5*IQR)) | (itImp_mid[iIt] > (Q3 + 1.5*IQR))]
  itImp_mid = itImp_mid.drop(train_delout.index, axis = 0)

더미변수 추가


In [9]:
def busyHourGen(data, col):
  lst = data[col]
  lst_ = []
  for i in lst:
    if (6 < i < 10) or (16 < i < 20):
      lst_.append(1)
    else:
      lst_.append(0)
  data['busy_hour'] = lst_
  return data

an = busyHourGen(itImp_mid, 'hour') # an = busy_hour 추가된 데이터셋

훈련용 검증용 변수 분리

In [11]:
# Filter rows with missing price values
# X = an.drop(['count'], axis=1)
X = an[an.columns.difference(['count', 'hour_bef_visibility'])]
X = np.column_stack((X['hour']**5, X['hour_bef_temperature']**4, X))
y = an['count']

train_X, val_X, train_y, val_y = train_test_split(X, y, shuffle = True, random_state=2021, test_size=0.33)

In [12]:
def objective_extratree(trial):
  params_et = {
      "n_estimators": trial.suggest_int("n_estimators", 500, 700, step=1),
      "max_depth": trial.suggest_int("max_depth", 10, 20),
      "min_samples_split": trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features": trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4, 5, 6]),
      "warm_start": trial.suggest_categorical("warm_start", [True, False]),
      "random_state": 2021
  }

  X = an[an.columns.difference(['count', 'hour_bef_visibility'])]
  y = an['count']
  train_X, val_X, train_y, val_y = train_test_split(X, y, shuffle = True, random_state=2021, test_size=0.2)

  model = ExtraTreesRegressor(**params_et)
  model.fit(train_X, train_y)

  pred = model.predict(val_X)
  rmse = np.sqrt(mean_squared_error(val_y, pred))

  return rmse

In [13]:
sampler = TPESampler(seed=2021)
study = optuna.create_study(
    study_name="et_optimizer",
    direction="minimize", 
    sampler=sampler,
    )

[I 2022-01-21 12:30:25,028] A new study created in memory with name: et_optimizer


In [18]:
#독립변수, 종속변수 설정
X = an[an.columns.difference(['id', 'count', 'hour_bef_visibility'])]
colls = X.columns.tolist()
X = np.column_stack((X['hour']**5, X['hour_bef_temperature']**4, X))
y = an['count']

#테스트셋 전처리(결측치, 더미 변수 추가)
it_test = test[test.columns.difference(['id', 'hour_bef_visibility'])].copy()
it_test = busyHourGen(it_test, 'hour')
it_test = it_test[colls]
X_test = np.column_stack((it_test['hour']**5, it_test['hour_bef_temperature']**4, it_test))
X_test = IterativeImputer(random_state=2021).fit_transform(X_test)

# ExtraTreesRegressor
best_reg = ExtraTreesRegressor(n_estimators=620, max_depth=19, warm_start=True, random_state=2021)
best_reg.fit(X, y)
predicts = best_reg.predict(X_test)

submission['count'] = predicts
submission.to_csv('final.csv', index=False)

submission['count'] = predicts.round(0)
submission.to_csv('final_rounded.csv', index=False)

In [14]:
study.optimize(objective_extratree, n_trials=100)

[I 2022-01-21 12:30:29,380] Trial 0 finished with value: 40.366999212472514 and parameters: {'n_estimators': 621, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 0 with value: 40.366999212472514.
[I 2022-01-21 12:30:30,321] Trial 1 finished with value: 40.09807204778074 and parameters: {'n_estimators': 693, 'max_depth': 16, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'warm_start': True}. Best is trial 1 with value: 40.09807204778074.
[I 2022-01-21 12:30:31,191] Trial 2 finished with value: 39.84898582200252 and parameters: {'n_estimators': 617, 'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5, 'warm_start': False}. Best is trial 2 with value: 39.84898582200252.
[I 2022-01-21 12:30:31,984] Trial 3 finished with value: 39.75870680335304 and parameters: {'n_estimators': 560, 'max_depth': 19, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5,

변수 중요도 확인

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance

xgbr_all = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, objective='reg:squarederror', random_state=1)
xgbr_all.fit(train_X, train_y)
pred = xgbr_all.predict(val_X)

rmse = np.sqrt(mean_squared_error(val_y, pred))
print(rmse)

fig, ax = plt.subplots(figsize=(8, 6))
plot_importance(xgbr_all, ax=ax)

In [ ]:
df_train = pd.DataFrame(itImp_mid, columns=col_name)
from pycaret.regression import *
reg = setup(df_train, target = 'count', train_size=0.8)

In [ ]:
best = compare_models(sort = 'RMSE')

In [ ]:


#bike_data['hour_bef_ozone'] = np.log(bike_data['hour_bef_ozone']+1)
#bike_data['hour_bef_pm10'] = np.log(bike_data['hour_bef_pm10']+1)
#bike_data['hour_bef_pm2.5'] = np.log(bike_data['hour_bef_pm2.5']+1)
 #= bike_data[['hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5']].apply(lambda x: np.log(x+1))

f, axes = plt.subplots(4, 3, figsize = (10, 10), constrained_layout=True)
for i, feature in enumerate(bike_data.columns):
  sns.distplot(bike_data[feature], ax = axes[i//3, i%3])
  print("bike_data[", feature, "]의 왜도: ", bike_data[feature].skew())

plt.show()

# fb = fb[['hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5']].apply(lambda x: np.log(x+1))

##val data 분리

In [ ]:
from sklearn.model_selection import train_test_split

# Filter rows with missing price values
filtered_bike_data = bike_data.dropna(axis=0)
X = filtered_bike_data.drop(['count'], axis=1)
y = filtered_bike_data['count']

train_X, val_X, train_y, val_y = train_test_split(X, y, shuffle = True, random_state=1, test_size=0.2)

#2. 탐색적 자료분석(EDA)
##기존 데이터 확인

In [ ]:
import matplotlib.pyplot as plt

bike_data.isna().sum()
bike_data.info()
#bike_data.groupby('hour').mean()['hour_bef_windspeed'].plot()
#bike_data[bike_data['hour_bef_temperature'].isna()]
# hour 기준으로 평균을 내고 카운트를 출력
#bike_data.groupby('hour').mean()['count'].plot()
plt.plot(bike_data.groupby('hour').mean()['count'], 'o-')
plt.grid()

In [ ]:
#print(bike_data.corr())
plt.figure(figsize=(10, 10))
sns.heatmap(bike_data.corr(), annot=True)

##모델 정의 및 학습

In [ ]:
added_test = busyHourGen(itImp_test, 'hour') # added_test = busy_hour 추가된 데이터셋
added_test.describe()

In [ ]:
model = RandomForestRegressor()
#model.fit(train_X, train_y)
model.fit(X, y)
test.fillna(0,inplace = True)


pred = model.predict(test)

#submission['count'] = pred
submission.to_csv('베이스라인_테스트만0.csv',index = False)
# plt.scatter(val_y, pred)
# plt.show()
# #print(pred)
# print(mean_squared_error(val_y, pred)**0.5)

##DNN을 위한 데이터 정규화

In [ ]:
# cnn을 위한 데이터 정규화
# min_X = train_X.min()
# max_X = train_X.max()
# min_y = train_y.min()
# max_y = train_y.max()
# train_X = (train_X - min_X) / (max_X - min_X)
# train_y = (train_y - min_y) / (max_y - min_y)

print(train_X[:5])
print(val_X[:5])
print(train_y[:5])
print(val_y[:5])

min_X = train_X.min()
max_X = train_X.max()
def normalize_X(X):
  return (X - min_X) / (max_X - min_X)

min_y = train_y.min()
max_y = train_y.max()
def normalize_y(y):
  return (y - min_y) / (max_y - min_y)

def denormalize_y(y):
  return (max_y - min_y) * y + min_y

norm_train_X = normalize_X(train_X)
norm_val_X = normalize_X(val_X)
norm_train_y = normalize_y(train_y)
norm_val_y = normalize_y(val_y)

print(norm_train_X[:5])
print(norm_val_X[:5])
print(norm_train_y[:5])
print(norm_val_y[:5])

In [ ]:
model = RandomForestRegressor()
model.fit(norm_train_X, norm_train_y)
pred = model.predict(norm_val_X)
denorm_pred = denormalize_y(pred)

plt.scatter(val_y, pred)
plt.show()
#print(pred)
print(mean_squared_error(val_y, denorm_pred)**0.5)

In [ ]:
import pandas as pd

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense

model = keras.Sequential()
model.add(Dense(1000, activation='relu', input_shape=(9,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile(optimizer="SGD", loss="mse", metrics=["mape"])
model.summary()

history = model.fit(norm_train_X, norm_train_y, epochs=100, verbose=1, batch_size=16, validation_split=0.2)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

loss, mape = model.evaluate(norm_val_X, norm_val_y)
print("loss=", loss)
print("mape=", mape)

##학습된 모델로 예측 데이터 생성

In [ ]:
from sklearn.metrics import mean_squared_error

pred = model.predict(norm_val_X)

In [ ]:
denorm_pred = denormalize_y(pred)
print(denorm_pred[:5])
#denorm_norm_val_y = denormalize_y(norm_val_y)
#print(denorm_norm_val_y[:5])


In [ ]:
plt.scatter(val_y, denorm_pred)
plt.show()
#print(pred)
print(mean_squared_error(val_y, denorm_pred)**0.5)